In [376]:
%run ../config/config.py
#from config.config import *
import pandas as pd
import numpy as np
import debug
import cv2

In [377]:
image = cv2.imread(BLACK_YELLOW)
image_size = (SIZE,SIZE)
image = cv2.resize(image,image_size)

In [378]:
def _detect_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, THRESHOLD1, THRESHOLD2, EDGES)
    return edges

edges_image = _detect_edges(image)

In [379]:
def _generate_edges_lines(canny):
    
    lines = cv2.HoughLines(canny, 
                            rho = RHO_RESOLUTION, 
                            theta =  THETA_RESOLUTION, 
                            threshold = HOUGH_THRESHOLD)
    
    #Format dataset
    lines = lines.reshape(len(lines),2)
    
    coor_lines = []
    for line in lines:
        rho,theta = line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 +DISTANCE*(-b))
        y1 = int(y0 +DISTANCE*(a))
        x2 = int(x0 -DISTANCE*(-b))
        y2 = int(y0 -DISTANCE*(a))   
        coor_lines.append(((x1,y1),(x2,y2)))
    
    return np.array(coor_lines)
    
lines = _generate_edges_lines(edges_image)

In [380]:
from sklearn.cluster import KMeans

def _get_lines_means(points, direction, n_clusters = 9):
    
    means = []

    for lines in points:
    
        kmeans = KMeans(n_clusters = n_clusters, n_init=10)
        kmeans.fit(lines)
        labels = kmeans.labels_
    
        unique_labels = list(range(n_clusters))
    
        cluster_means = []
    
        for label in unique_labels: 
            cluster_data = lines[labels == label] 
            cluster_mean = np.round(sum(cluster_data)/len(cluster_data))  
            cluster_means.append(cluster_mean)
        
        cluster_means = np.array(cluster_means)
        sorted_index = np.argsort(cluster_means[:, direction])
        
        means.append(cluster_means[sorted_index])
 
    return means


In [381]:


def _get_vertical_averages(lines):
    
    vertical_lines = lines[lines[:,0,1] > SIZE]
    
    initial_points = vertical_lines[:,1][vertical_lines[:,1,1] < -SIZE]
    final_points = vertical_lines[:,0][vertical_lines[:,0,1] > SIZE] 
    
    averages = _get_lines_means((initial_points, final_points), VERTICAL)
    
    return np.stack(averages, axis=1)

def _get_horizontal_averages(lines):

    horizontal_lines = lines[lines[:,1,0] > SIZE]
    
    initial_points = horizontal_lines[:,0][horizontal_lines[:,0,0] < -SIZE]
    final_points = horizontal_lines[:,1][horizontal_lines[:,1,0] > SIZE]
    
    averages = _get_lines_means((initial_points, final_points), HORIZONTAL)
    
    return np.stack(averages, axis=1)
    
vertical_lines = _get_vertical_averages(lines).astype(int)
horizontal_lines = _get_horizontal_averages(lines).astype(int)

lines = np.concatenate((vertical_lines,horizontal_lines), axis=0)

debug.lines(image,lines, size = 3);


In [382]:
def _detect_inner_corners(lines, corners = MAX_CORNERS):
    
    black_image = np.zeros((SIZE, SIZE, EDGES), dtype=np.uint8)
    debug.lines(black_image,lines, size = 3);
    black_image = _detect_edges(black_image)
    points = cv2.goodFeaturesToTrack(black_image, corners, QUALITY, MIN_DISTANCE)
    return points.reshape(len(points),2)
    
points =  _detect_inner_corners(lines)
debug.points(image,points, color = BLUE, size = 7);


In [383]:
"""import itertools

def line_intersections(line1, line2):
    x1, y1 = line1[0]
    x2, y2 = line1[1]
    x3, y3 = line2[0]
    x4, y4 = line2[1]

    # Calculate the denominator for the intersection point calculation
    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if  0 in denominator:
        return None  # Lines are parallel and do not intersect

    # Calculate the intersection point
    px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
    py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator

    return (px, py)

def line_intersection(lines):

    lines = list(itertools.combinations(lines, 2))
    intersections = []

    for i in range(len(lines)):
        intersection = line_intersections(lines[0], lines[1])
        if intersection is not None:
                intersections.append(intersection)
            
    return np.array(points)



#print(lines)
points = line_intersection(lines)

print(points)

lower_limit = -1000
upper_limit = 1000
#points = points.astype(int)
# Create a mask to filter points within the range
#mask = (points[:, 0] >= lower_limit) & (points[:, 0] <= upper_limit) & (points[:, 1] >= lower_limit) & (points[:, 1] <= upper_limit)

#points = points[mask]

print(len(points))
debug.points(image,points, color = (0,255,0), size = 7);

#print(len(sorted_intersections))"""

'import itertools\n\ndef line_intersections(line1, line2):\n    x1, y1 = line1[0]\n    x2, y2 = line1[1]\n    x3, y3 = line2[0]\n    x4, y4 = line2[1]\n\n    # Calculate the denominator for the intersection point calculation\n    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)\n    if  0 in denominator:\n        return None  # Lines are parallel and do not intersect\n\n    # Calculate the intersection point\n    px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator\n    py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator\n\n    return (px, py)\n\ndef line_intersection(lines):\n\n    lines = list(itertools.combinations(lines, 2))\n    intersections = []\n\n    for i in range(len(lines)):\n        intersection = line_intersections(lines[0], lines[1])\n        if intersection is not None:\n                intersections.append(intersection)\n            \n    return np.array(points)\n\n\n\n#print(lines)\npoints

In [384]:
def _get_middle_squares(lines):

    final = []
    print(lines)

    for i in range(0,8):
        line1 = lines[i]
        line2 = lines[i + 1]

        a = int((line1[0][0] + line2[0][0]) / 2)
        b = int((line1[0][1] + line2[0][1]) / 2)
        c = int((line1[1][0] + line2[1][0]) / 2)
        d = int((line1[1][1] + line2[1][1]) / 2)

        final_line = [[a,b], [c,d]]

        final.append(final_line)

    for i in range(9,17):
        line1 = lines[i]
        line2 = lines[i + 1]

        a = int((line1[0][0] + line2[0][0]) / 2)
        b = int((line1[0][1] + line2[0][1]) / 2)
        c = int((line1[1][0] + line2[1][0]) / 2)
        d = int((line1[1][1] + line2[1][1]) / 2)

        final_line = [[a,b], [c,d]]

        final.append(final_line)

    corners = _detect_inner_corners(final, 64)
    return corners

final = _get_middle_squares(lines)
print(final)
debug.points(image, final, color = GREEN, size = 5)

[[[   84 -1399]
  [   51  1400]]

 [[  170 -1397]
  [  120  1402]]

 [[  235 -1397]
  [  203  1402]]

 [[  308 -1397]
  [  284  1402]]

 [[  384 -1396]
  [  360  1403]]

 [[  458 -1396]
  [  434  1404]]

 [[  535 -1394]
  [  506  1405]]

 [[  607 -1394]
  [  576  1408]]

 [[  666 -1393]
  [  653  1404]]

 [[-1400     8]
  [ 1398    72]]

 [[-1402    72]
  [ 1397   140]]

 [[-1402   153]
  [ 1397   205]]

 [[-1401   247]
  [ 1398   270]]

 [[-1403   330]
  [ 1397   351]]

 [[-1401   401]
  [ 1393   437]]

 [[-1404   500]
  [ 1402   510]]

 [[-1394   586]
  [ 1401   567]]

 [[-1393   649]
  [ 1403   636]]]
[[246. 603.]
 [168. 603.]
 [399. 602.]
 [324. 602.]
 [615. 601.]
 [545. 601.]
 [472. 601.]
 [545. 535.]
 [473. 535.]
 [399. 535.]
 [324. 535.]
 [247. 535.]
 [169. 535.]
 [ 93. 535.]
 [616. 534.]
 [616. 462.]
 [546. 461.]
 [474. 460.]
 [400. 460.]
 [325. 459.]
 [247. 459.]
 [170. 458.]
 [ 94. 457.]
 [617. 381.]
 [547. 380.]
 [475. 379.]
 [401. 379.]
 [326. 378.]
 [248. 377.]
 [171. 376.

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [385]:

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()